In [88]:
import numpy as np
import pandas as pd
from scipy import stats

# Задача 1
**Провести дисперсионный анализ для определения того, есть ли различия среднего роста среди взрослых футболистов, хоккеистов и штангистов. Даны значения роста в трех группах случайно выбранных спортсменов: Футболисты: 173, 175, 180, 178, 177, 185, 183, 182. Хоккеисты: 177, 179, 180, 188, 177, 172, 171, 184, 180. Штангисты: 172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170.**

Рост человека в популяции распределяется по нормальному закону, исследуется влияние одного фактора, поэтому для решения задачи применим однофакторный дисперсионый анализ. В качестве качественного показателя выступает вид спорта, в качестве количественного показателя вид — спорта. Количество уровней — 3 ($k=3$). Сформулируем гипотезы:

$ H_0: \mu_1 = \mu_2 = \mu_3 \newline H_1 \begin{cases} H_{1.1}: \mu_1 \neq \mu_2 \\ H_{1.2}: \mu_1 \neq \mu_3 \\ H_{1.3}: \mu_2 \neq \mu_3\end{cases}$

При сравнении более двух групп ($k > 2$) сравнение производися с помощью критерия Фишера F, при сравнении двух групп ($k = 2$) используется критерий Стьюдента t. Критерий Фишера вычисляется по формуле:

$F_H = \frac{\sigma_ф^2}{\sigma_{ост}^2}$, где ${\sigma_ф^2}$ — факторная дисперсия (межгрупповая дисперсия, дисперсия средних $\mu$), а ${\sigma_{ост}^2}$ — остаточная дисперсия (внутригрупповая, необъясненная дисперсия внутри группы).

Факторная дисперсия рассчитывается по формуле ${\sigma_ф^2} = \frac{s_ф^2}{k - 1} = \frac{\sum_{i = 1}^k(\bar y_i - \bar Y)^2 \cdot n_i}{k - 1}$, где ${s_ф^2}$ — сумма квадратов отклонений средних по выборке от среднего по всем выборкам, $\bar y_i$ — среднее арифметическое $i$-той выборки, $\bar Y$ — среднее арифметическое по всем выборкам, $n_i$ — размер $i$-той выборки.

Остаточная дисперсия рассчитывается по формуле ${\sigma_{ост}^2} = \frac{s_{ост}^2}{n - k} = \frac{\sum_{i = 1}^k{\sum_{j=1}^{n_j}(y_{ij} - \bar y_i)^2}}{n - k}$, где ${s_{ост}^2}$ — сумма сумм квадратов отклонений значений от средних по выборке, $\bar y_{ij}$ — ${j}$-тое значение $i$-той выборки, $\bar y_i$ — среднее арифметическое $i$-той выборки.

Расчет будем производить для $\alpha = 0.05$. Критическое значение криитерия Фишера берется из соответствующей таблицы или с помощю функции модуля scipy.stats.

In [89]:
alpha = 0.05
data = []
data.append(np.array([173, 175, 180, 178, 177, 185, 183, 182]))                 # football_playes
data.append(np.array([177, 179, 180, 188, 177, 172, 171, 184, 180]))            # hockey_players
data.append(np.array([172, 173, 169, 177, 166, 180, 178, 177, 172, 166, 170]))  # weightlifter
data.append(np.array([val for arr in data for val in arr]))
k = len(data) - 1
n = len(data[k])

mean = [None for _ in range(k + 1)]
mean[k] = data[k].mean()
s_fac = 0
s_ost = 0
for i in range(k):
    mean[i] = data[i].mean()
    s_fac += (mean[i] - mean [k]) ** 2 * len(data[i])
    for j in range (len(data[i])):
        s_ost += (data[i][j] - mean [i]) ** 2
sigma_fac = s_fac / (k - 1)
sigma_ost = s_ost / (n - k)
fischer_scor = sigma_fac / sigma_ost
fischer_crit = stats.f.ppf(q = 1 - alpha, dfn = k - 1, dfd = n - k)
if fischer_scor < fischer_crit:
    print (f'Расчетный критерий Фишера {"{0:.4f}".format(fischer_scor)} ' + \
           f'меньше критического табличного значения {"{0:.4f}".format(fischer_crit)}.\n' + \
            'Принимается гипотеза H0. Статистически значимых различий не обнаружено.')
else:
    print (f'Расчетный критерий Фишера {"{0:.4f}".format(fischer_scor)} ' + \
           f'больше критического табличного значения {"{0:.4f}".format(fischer_crit)}.\n' + \
            'Принимается гипотеза H1. Обнаружены статистически значимые различия.')

Расчетный критерий Фишера 5.5001 больше критического табличного значения 3.3852.
Принимается гипотеза H1. Обнаружены статистически значимые различия.


Все эти вычисления можно упростить с помощью функций модуля scipy.stats — f_oneway.

In [90]:
fischer_scor, pvalue = stats.f_oneway(data[0], data[1], data[2])
if pvalue > alpha:
    print (f'Расчетный критерий Фишера {"{0:.4f}".format(fischer_scor)} ' + \
           f'меньше критического табличного значения.\n' + \
           f'p-value {"{0:.4f}".format(pvalue)} больше заданной alpha {"{0:.4f}".format(alpha)}.\n' + \
            'Принимается гипотеза H0. Статистически значимых различий не обнаружено.')
else:
    print (f'Расчетный критерий Фишера {"{0:.4f}".format(fischer_scor)} ' + \
           f'больше критического табличного значения.\n' + \
           f'p-value {"{0:.4f}".format(pvalue)} меньше заданной alpha {"{0:.4f}".format(alpha)}.\n' + \
            'Принимается гипотеза H1. Обнаружены статистически значимые различия.')

Расчетный критерий Фишера 5.5001 больше критического табличного значения.
p-value 0.0105 меньше заданной alpha 0.0500.
Принимается гипотеза H1. Обнаружены статистически значимые различия.


*Ответ: обнаружены статистически значимые различия. Рост спортсменов в различных видах спорта различный.*